In [1]:
from typing import Dict, Optional, Tuple
from overrides import overrides

import os
import copy
import time
import torch
import random
import numpy as np
import pandas as pd
from scripts.row_pop import RowPop

from torch.autograd import Variable
from allennlp.nn import util
from allennlp.data import Vocabulary
from allennlp.models.model import Model
from allennlp.modules.token_embedders import Embedding
from allennlp.modules import FeedForward, TextFieldEmbedder, Seq2VecEncoder
from allennlp.nn import InitializerApplicator, RegularizerApplicator
from allennlp.training.metrics.categorical_accuracy import CategoricalAccuracy
# from allennlp.modules.token_embedders import PretrainedBertEmbedder
from table_embedder.models.lib.bert_token_embedder import PretrainedBertEmbedder
from allennlp.models.archival import load_archive

from table_embedder.models.embedder_util import TableUtil
from table_embedder.models.lib.stacked_self_attention import StackedSelfAttentionEncoder
# from lib.stacked_self_attention import StackedSelfAttentionEncoder

from table_embedder.models.cache_util import CacheUtil
#from .lib.masked_ff import MaskedFeedForward
# torch.set_default_tensor_type(torch.DoubleTensor)


from table_embedder.models.cache_util import CacheUtil

from torch import nn
from torch import tensor
import pdb

from torch_util.hypers_base import HypersBase

/work/arnaik_umass_edu/.conda/envs/table_emb_py37/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/work/arnaik_umass_edu/.conda/envs/table_emb_py37/lib/python3.7/site-packages/apex/pyprof/__init__.py:5: FutureWarning: pyprof will be removed by the end of June, 2022
  warnings.warn("pyprof will be removed by the end of June, 2022", FutureWarning)


In [2]:
class TableEmbedderHypers(HypersBase):
    def __init__(self):
        super().__init__()
        self.num_max_row_pos = 31
        self.num_max_col_pos = 25
        self.n_classes = 127656  # FIXME: we'll update this, since I can't reproduce
        self.n_seed_rows = 2
        # where we have clscol.npy  clsrow.npy
        self.saved_table_cls_dir = '/work/arnaik_umass_edu/tabbie/data'  # OR on CCC: '/dccstor/few-shot-rel/TableAugmentation/tabbie/data'
        self.model_file = '/work/arnaik_umass_edu/model_named.pt'
        
    def _post_init(self):
        super()._post_init()
        self.per_gpu_train_batch_size = 1
        if self.full_train_batch_size < self.world_size or self.full_train_batch_size % self.world_size != 0:
            raise ValueError(f'full_train_batch_size ({self.full_train_batch_size} '
                             f'must be a multiple of world_size ({self.world_size})')
        self.gradient_accumulation_steps = self.full_train_batch_size // self.world_size        

In [3]:
from torch import tensor

In [4]:
table_info = [{'table_id': '1438042981460.12__CC-MAIN-20150728002301-00003-ip-10-236-191-2.ec2.internal__1387'
               , 'num_rows': 2
               , 'num_cols': 7
               , 'header': ['size', 'us', 'type', 'china', 'gaa', 'eu', 'years']
               , 'cell_labels': None
               , 'col_labels': None
               , 'table_labels': None
               , 'table_data_raw': [['xs', '6/7', '9.125', '235', '23', '36-37', '4']
                                    , ['s', '7/8', '9.5', '245', '24', '38-39', '5']]
               , 'table': [['size', 'us', 'type', 'china', 'gaa', 'eu', 'years']
                           , ['xs', '6/7', '9.125', '235', '23', '36-37', '4']
                           , ['s', '7/8', '9.5', '245', '24', '38-39', '5']]}]

In [5]:
indexed_headers = {'bert': {'token_ids': tensor([[[  101,  2946,   102],
         [  101,  2149,   102],
         [  101,  2828,   102],
         [  101,  2859,   102],
         [  101, 19930,   102],
         [  101,  7327,   102],
         [  101,  2086,   102]]], device='cuda:0')
, 'mask': tensor([[[True, True, True],
         [True, True, True],
         [True, True, True],
         [True, True, True],
         [True, True, True],
         [True, True, True],
         [True, True, True]]], device='cuda:0')
, 'type_ids': tensor([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]], device='cuda:0')}}

In [6]:
indexed_cells = {'bert': {'token_ids': tensor([[[[  101,  1060,  2015,   102,     0],
          [  101,  1020,  1013,  1021,   102],
          [  101,  1023,  1012,  8732,   102],
          [  101, 17825,   102,     0,     0],
          [  101,  2603,   102,     0,     0],
          [  101,  4029,  1011,  4261,   102],
          [  101,  1018,   102,     0,     0]],

         [[  101,  1055,   102,     0,     0],
          [  101,  1021,  1013,  1022,   102],
          [  101,  1023,  1012,  1019,   102],
          [  101, 21005,   102,     0,     0],
          [  101,  2484,   102,     0,     0],
          [  101,  4229,  1011,  4464,   102],
          [  101,  1019,   102,     0,     0]]]], device='cuda:0')
, 'mask': tensor([[[[ True,  True,  True,  True, False],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True, False, False],
          [ True,  True,  True, False, False],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True, False, False]],

         [[ True,  True,  True, False, False],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True, False, False],
          [ True,  True,  True, False, False],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True, False, False]]]], device='cuda:0')
, 'type_ids': tensor([[[[0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]],

         [[0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]]]], device='cuda:0')}}

In [7]:
type(table_info), type(indexed_headers), type(indexed_cells)

(list, dict, dict)

In [11]:
hypers = TableEmbedderHypers()
model = TableEmbedderTorch(hypers).to('cuda:0')

In [12]:
loss, prob = model(table_info,indexed_headers, indexed_cells, None)

KeyError: 'label_idx'

In [10]:
class TableEmbedder(torch.nn.Module):

    def __init__(self, hypers: TableEmbedderHypers) -> None:
        super().__init__()
        self.hypers = hypers
        
        self.row_pos_embedding = Embedding(num_embeddings=self.hypers.num_max_row_pos, embedding_dim=768)
        self.col_pos_embedding = Embedding(num_embeddings=self.hypers.num_max_col_pos, embedding_dim=768)        
        
        # TODO: Create the feedforward layer for row population
        self.top_feedforward = nn.Linear(768*self.hypers.n_seed_rows, self.hypers.n_classes)
        self.Dropout = nn.Dropout(p = 0.0) # At the moment Dropout not used as only one-layer network

        # # self.row_feedforward = row_feedforward # NOTE : Not used in forward pass
        # self.feedforward = feedforward # NOTE : Feedforward layer but not used in forward pass
        # self.compose_ff = compose_ff # NOTE : Feedforward layer but not used in forward pass
        
        self.bert_embedder = PretrainedBertEmbedder(pretrained_model='bert-base-uncased', top_layer_only=True)
        self.transformer_col1 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_col2 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_col3 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_col4 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_col5 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_col6 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_col7 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_col8 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_col9 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_col10 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_col11 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_col12 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)

        self.transformer_row1 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_row2 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_row3 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_row4 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_row5 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_row6 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_row7 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_row8 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_row9 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_row10 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_row11 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_row12 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.loss = torch.nn.BCEWithLogitsLoss()
        self.loss_func = torch.nn.CrossEntropyLoss()

        self.cache_usage = None

        # TODO: try these as plain parameters, initialized from the saved clscol and clsrow embeddings
        self.cls_col = np.load(os.path.join(hypers.saved_table_cls_dir, 'clscol.npy'))
        self.cls_row = np.load(os.path.join(hypers.saved_table_cls_dir, 'clsrow.npy'))

        self.opt_level = 'O0'

        ## TODO : Updated the 'getenv'
        if self.cache_usage is not None:
            self.cache_util = CacheUtil(self.cache_usage, os.getenv("cell_db_path"))
        else:
            self.cache_util = None

        self.init_weight() # As feedforward layer is not present in the archieved model rather than using 'load_state_dict' still updating the model via a function
        
        # # TODO : Check what the load_label function does
        # self.label = RowPop.load_label(os.getenv('label_path'), key='index')

    def init_weight(self):
        model_parameters = dict(self.named_parameters())
        archived_parameters = torch.load(self.hypers.model_file) # Archieved weights of feedforward layers discarded.
        for name, weights in archived_parameters.items():
            if name in model_parameters:
                new_weights = weights.data
                model_parameters[name].data.copy_(new_weights)

    def get_tabemb(self, bert_header, bert_data, n_rows, n_cols, bs, table_mask, nrows, ncols):
        row_pos_ids = torch.arange(0, self.hypers.num_max_row_pos, device=self.device, dtype=torch.long)
        col_pos_ids = torch.arange(0, self.hypers.num_max_col_pos, device=self.device, dtype=torch.long)

        n_rows += 1  # row CLS
        n_cols += 1  # col CLS
        cls_col = torch.from_numpy(copy.deepcopy(self.cls_col)).to(device=self.device)
        cls_row = torch.from_numpy(copy.deepcopy(self.cls_row)).to(device=self.device)
        row_pos_embs = self.row_pos_embedding(row_pos_ids[:n_rows+1])
        col_pos_embs = self.col_pos_embedding(col_pos_ids[:n_cols])

        for i in range(1, 13):
            transformer_row = getattr(self, 'transformer_row{}'.format(str(i)))
            transformer_col = getattr(self, 'transformer_col{}'.format(str(i)))
            if i == 1:
                bert_data = TableUtil.add_cls_tokens(bert_header, bert_data, cls_row, cls_col, bs, n_rows, n_cols)
                bert_data += row_pos_embs.expand((bs, n_cols, n_rows + 1, 768)).permute(0, 2, 1, 3).expand_as(bert_data)
                bert_data += col_pos_embs.expand((bs, n_rows + 1, n_cols, 768)).expand_as(bert_data)
                # table_mask = TableUtil.add_cls_mask(table_mask, table_info, bs, n_rows, n_cols, self.device)
                table_mask = TableUtil.add_cls_mask(table_mask, bs, n_rows, n_cols, self.device, nrows, ncols)
                col_embs = TableUtil.get_col_embs(bert_data, bs, n_rows, n_cols, table_mask, transformer_col, self.opt_level)
                row_embs = TableUtil.get_row_embs(bert_data, bs, n_rows, n_cols, table_mask, transformer_row, self.opt_level)
            else:
                row_embs = TableUtil.get_row_embs(ave_embs, bs, n_rows, n_cols, table_mask, transformer_row, self.opt_level)
                col_embs = TableUtil.get_col_embs(ave_embs, bs, n_rows, n_cols, table_mask, transformer_col, self.opt_level)
            ave_embs = (row_embs + col_embs) / 2.0
        return row_embs, col_embs, n_rows, n_cols

    # @staticmethod
    # def get_labels(table_info):
    #     labels = []
    #     labels_1d = []
    #     for one_info in table_info:
    #         row_labels = list(set(copy.deepcopy(one_info['label_idx'])))
    #         if 0 in row_labels:
    #             row_labels.remove(0)
    #         labels.append(row_labels)
    #         for elem in row_labels:
    #             labels_1d.append(elem)
    #     return labels_1d, labels

    def pred_prob(self, cell_embs, labels, mask):
        out_prob = self.top_feedforward(cell_embs, mask)
        out_prob_1d = []
        for k, one_prob in enumerate(out_prob):
            # out_prob_1d.append(one_prob.expand(len(labels[k]), self.n_classes))
            out_prob_1d.append(one_prob.expand(len(labels[k]), out_prob.shape[1]))
        out_prob_1d = torch.cat(out_prob_1d, dim=0)
        return out_prob_1d, out_prob

    @staticmethod
    def add_metadata(table_info, output_dict, pred_labels, pred_labels_name):
        data_dict = {'pred_labels': pred_labels, 'pred_labels_name': pred_labels_name}
        for one_info in table_info:
            for k, v in one_info.items():
                data_dict[k] = data_dict.get(k, [])
                data_dict[k].append(v)
        output_dict.update(data_dict)
        return output_dict

    def get_pred_labels(self, out_prob, labels, top_k=-1):
        pred_labels = []
        pred_labels_name = []
        for k, row_labels in enumerate(labels):
            n_pred = len(row_labels) if top_k == -1 else top_k
            pred_row_labels = out_prob[k][1:].argsort(dim=0, descending=True)[:n_pred].cpu().numpy()  # out_prob[0]: blank header
            pred_row_labels = [elem+1 for elem in pred_row_labels]  # add idx to 1 (for out_prob[0])
            pred_labels.append(pred_row_labels)
            pred_labels_name.append([self.label[elem] for elem in pred_row_labels])
        return pred_labels, pred_labels_name

    def sample_labels(self, labels_1d):
        # sampled label
        mask_bool = torch.cuda.FloatTensor(self.n_classes).uniform_() > 0.8
        mask = torch.tensor(mask_bool, dtype=torch.int)
        mask[labels_1d] = 2

        # target label
        new_labels = mask[mask!=0]
        new_labels[new_labels==1] = 0
        new_labels[new_labels==2] = 1

        # old_new label map
        old_idx = (mask==2).nonzero()
        new_idx = (new_labels==1).nonzero()
        idx_map = {}
        for k, idx in enumerate(old_idx):
            idx_map[int(idx)] = int(new_idx[k])
        new_labels_1d = torch.autograd.Variable(labels_1d.clone())
        for k, idx in enumerate(labels_1d):
            new_labels_1d[k] = idx_map[int(idx)]
        return mask, new_labels_1d

    def validate_seed_rows(self, table_info):
        for one_info in table_info:
            if one_info['num_rows'] > self.n_seed_rows:
                raise ValueError('invalid num rows')

    def get_meta(self, table_info):
        nrows = [one_info['num_rows'] for one_info in table_info]
        ncols = [one_info['num_cols'] for one_info in table_info]
        tids = [one_info['table_id'] for one_info in table_info]
        return nrows, ncols, tids

    @overrides
    def forward(self,
                table_info: Dict[str, str],
                indexed_headers: Dict[str, torch.LongTensor],
                indexed_cells: Dict[str, torch.LongTensor],
                labels: Optional[torch.FloatTensor]) -> Tuple[Optional[torch.Tensor], torch.Tensor]:
        """

        :param table_info: see data/table_info.txt for example
        :param indexed_headers:  see data/indexed_headers.txt for example
        :param indexed_cells: see data/indexed_headers.txt for example
        :param labels: 0 or 1 tensor with shape: bs x self.hypers.n_classes
        :return: loss and output probabilities (for each possible header)
        """
        
        # initialize
        self.bert_embedder.eval()
        # self.validate_seed_rows(table_info)
        bs, n_rows, n_cols = TableUtil.get_max_row_col(table_info)
        nrows, ncols, tids = self.get_meta(table_info)
        table_mask = TableUtil.get_table_mask(table_info, bs, n_rows, n_cols, self.device)

        # pred prob
        bert_header, bert_cell = TableUtil.get_bert_emb(indexed_headers, indexed_cells, table_info, bs, n_rows, n_cols, self.cache_usage, self.bert_embedder, self.cache_util, self.device)
        row_embs, col_embs, n_rows_cls, n_cols_cls = self.get_tabemb(bert_header, bert_cell, n_rows, n_cols, bs, table_mask, nrows, ncols)
        
        cell_embs = (col_embs[:, 0, 1, :] + row_embs[:, 0, 1, :]) / 2.0

        # label/prob
#        labels_1d, labels = self.get_labels(table_info)
        labels = torch.LongTensor(labels).to(device=self.device)
        mask, sampled_labels = self.sample_labels(labels)

        if not self.training:
            mask = torch.ones(mask.shape, dtype=torch.int)
            sampled_labels_1d = labels_1d

        out_prob_1d, out_prob = self.pred_prob(cell_embs, labels, mask)

        # calc loss func
        # loss = self.loss_func(out_prob_1d[:, mask.nonzero().flatten()], sampled_labels_1d)
        loss = self.loss(out_prob_1d, sampled_labels_1d)

        # pred labels
        # pred_labels, pred_labels_name = self.get_pred_labels(out_prob, labels)
        # out_prob_cp = self.mod_out_prob(out_prob_1d, pred_labels)

        # if random.random() < 0.01:
        #     for k, one_info in enumerate(table_info):
        #         print()
        #         print(one_info['seed_col0'])
        #         print(one_info['label'])
        #         print(pred_labels_name[k])

        output_dict = {'loss': loss}

        if not self.training:
            pred_labels, pred_labels_name = self.get_pred_labels(out_prob, labels, top_k=500)
            output_dict = self.add_metadata(table_info, output_dict, pred_labels, pred_labels_name)
        return output_dict

In [11]:
from torch import tensor
import pdb

In [12]:
table_info = [{'table_id': '1438042981460.12__CC-MAIN-20150728002301-00003-ip-10-236-191-2.ec2.internal__1387'
                  , 'num_rows': 3
                  , 'num_cols': 2
                  , 'header': ['size', 'japan (cm)']
                  , 'cell_labels': None
                  , 'col_labels': None
                  , 'table_labels': None
                  , 'table_data_raw': [['6/7', '23']
        , ['s', '24']
        , ['m', '25']]
                  , 'table': [['size', 'japan (cm)']
        , ['6/7', '23']
        , ['s', '24']
        , ['m', '25']]},{'table_id': '1438042981460.12__CC-MAIN-20150728002301-00003-ip-10-236-191-2.ec2.internal__1387'
                  , 'num_rows': 3
                  , 'num_cols': 2
                  , 'header': ['size', 'japan (cm)']
                  , 'cell_labels': None
                  , 'col_labels': None
                  , 'table_labels': None
                  , 'table_data_raw': [['6/7', '23']
        , ['s', '24']
        , ['m', '25']]
                  , 'table': [['size', 'japan (cm)']
        , ['6/7', '23']
        , ['s', '24']
        , ['m', '25']]},{'table_id': '1438042981460.12__CC-MAIN-20150728002301-00003-ip-10-236-191-2.ec2.internal__1387'
                  , 'num_rows': 3
                  , 'num_cols': 2
                  , 'header': ['size', 'japan (cm)']
                  , 'cell_labels': None
                  , 'col_labels': None
                  , 'table_labels': None
                  , 'table_data_raw': [['6/7', '23']
        , ['s', '24']
        , ['m', '25']]
                  , 'table': [['size', 'japan (cm)']
        , ['6/7', '23']
        , ['s', '24']
        , ['m', '25']]}]

indexed_headers = {'bert': {'token_ids': tensor([[[101, 2946, 102, 0, 0, 0],
                                                  [101, 2900, 1006, 4642, 1007, 102]]
                                                 ,[[101, 2946, 102, 0, 0, 0],
                                                  [101, 2900, 1006, 4642, 1007, 102]]
                                                 ,[[101, 2946, 102, 0, 0, 0],
                                                  [101, 2900, 1006, 4642, 1007, 102]]], device='cuda:0')
    , 'mask': tensor([[[True, True, True, False, False, False],
                       [True, True, True, True, True, True]],
                     [[True, True, True, False, False, False],
                       [True, True, True, True, True, True]],
                     [[True, True, True, False, False, False],
                       [True, True, True, True, True, True]]], device='cuda:0')
    , 'type_ids': tensor([[[0, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0]],
                         [[0, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0]],
                         [[0, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0]]], device='cuda:0')}}

indexed_cells = {'bert': {'token_ids': tensor([[[[101, 1020, 1013, 1021, 102],[101, 2603, 102, 0, 0]],

                                                [[101, 1055, 102, 0, 0],
                                                 [101, 2484, 102, 0, 0]],

                                                [[101, 1049, 102, 0, 0],
                                                 [101, 2423, 102, 0, 0]]],
                                              [[[101, 1020, 1013, 1021, 102],[101, 2603, 102, 0, 0]],

                                                [[101, 1055, 102, 0, 0],
                                                 [101, 2484, 102, 0, 0]],

                                                [[101, 1049, 102, 0, 0],
                                                 [101, 2423, 102, 0, 0]]],
                                              [[[101, 1020, 1013, 1021, 102],[101, 2603, 102, 0, 0]],

                                                [[101, 1055, 102, 0, 0],
                                                 [101, 2484, 102, 0, 0]],

                                                [[101, 1049, 102, 0, 0],
                                                 [101, 2423, 102, 0, 0]]]], device='cuda:0')
    , 'mask': tensor([[[[True, True, True, True, True],
                        [True, True, True, False, False]],

                       [[True, True, True, False, False],
                        [True, True, True, False, False]],

                       [[True, True, True, False, False],
                        [True, True, True, False, False]]],
                     [[[True, True, True, True, True],
                        [True, True, True, False, False]],

                       [[True, True, True, False, False],
                        [True, True, True, False, False]],

                       [[True, True, True, False, False],
                        [True, True, True, False, False]]],
                     [[[True, True, True, True, True],
                        [True, True, True, False, False]],

                       [[True, True, True, False, False],
                        [True, True, True, False, False]],

                       [[True, True, True, False, False],
                        [True, True, True, False, False]]]], device='cuda:0')
    , 'type_ids': tensor([[[[0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0]],

                           [[0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0]],

                           [[0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0]]],
                         [[[0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0]],

                           [[0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0]],

                           [[0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0]]],
                         [[[0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0]],

                           [[0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0]],

                           [[0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0]]]], device='cuda:0')}}

In [20]:
## Sample Input
table_info = [{'table_id': 'table-0875-744', 'num_rows': 57, 'num_cols': 2, 'header': ['Edition', 'Year'], 'cell_lables': None, 'col_lables': None, 'table_lables': None, 'table_data_raw': [['1st', '1957'], ['2nd', '1958'], ['3rd', '1959'], ['4th', '1960'], ['5th', '1961'], ['6th', '1962'], ['7th', '1963'], ['8th', '1964'], ['9th', '1965'], ['10th', '1966'], ['11th', '1967'], ['12th', '1968'], ['13th', '1969'], ['14th', '1970'], ['15th', '1971'], ['16th', '1972'], ['17th', '1973'], ['18th', '1974'], ['19th', '1975'], ['20th', '1976'], ['21st', '1977'], ['22nd', '1978'], ['23rd', '1979'], ['24th', '1980'], ['25th', '1981'], ['26th', '1982'], ['27th', '1983'], ['28th', '1984'], ['29th', '1985'], ['30th', '1986'], ['31st', '1987'], ['32nd', '1988'], ['33rd', '1989'], ['34th', '1990'], ['35th', '1991'], ['36th', '1992'], ['37th', '1993'], ['38th', '1994'], ['39th', '1995'], ['40th', '1996'], ['41st', '1997'], ['42nd', '1998'], ['43rd', '1999'], ['44th', '2000'], ['45th', '2001'], ['46th', '2002'], ['47th', '2003'], ['48th', '2004'], ['49th', '2005'], ['50th', '2006'], ['—', '2007'], ['51st', '2008'], ['52nd', '2009'], ['53rd', '2010'], ['54th', '2011'], ['55th', '2012'], ['56th', '2013']], 'table': [['Edition', 'Year'], ['1st', '1957'], ['2nd', '1958'], ['3rd', '1959'], ['4th', '1960'], ['5th', '1961'], ['6th', '1962'], ['7th', '1963'], ['8th', '1964'], ['9th', '1965'], ['10th', '1966'], ['11th', '1967'], ['12th', '1968'], ['13th', '1969'], ['14th', '1970'], ['15th', '1971'], ['16th', '1972'], ['17th', '1973'], ['18th', '1974'], ['19th', '1975'], ['20th', '1976'], ['21st', '1977'], ['22nd', '1978'], ['23rd', '1979'], ['24th', '1980'], ['25th', '1981'], ['26th', '1982'], ['27th', '1983'], ['28th', '1984'], ['29th', '1985'], ['30th', '1986'], ['31st', '1987'], ['32nd', '1988'], ['33rd', '1989'], ['34th', '1990'], ['35th', '1991'], ['36th', '1992'], ['37th', '1993'], ['38th', '1994'], ['39th', '1995'], ['40th', '1996'], ['41st', '1997'], ['42nd', '1998'], ['43rd', '1999'], ['44th', '2000'], ['45th', '2001'], ['46th', '2002'], ['47th', '2003'], ['48th', '2004'], ['49th', '2005'], ['50th', '2006'], ['—', '2007'], ['51st', '2008'], ['52nd', '2009'], ['53rd', '2010'], ['54th', '2011'], ['55th', '2012'], ['56th', '2013']]}]

indexed_headers = {'bert': {'token_ids': tensor([[[3179],
         [2095]]], device='cuda:0'), 'mask': tensor([[[True],
         [True]]], device='cuda:0'), 'type_ids': tensor([[[0],
         [0]]], device='cuda:0')}}

indexed_cells = {'bert': {'token_ids': tensor([[[[ 3083,     0],
          [ 3890,     0]],

         [[ 3416,     0],
          [ 3845,     0]],

         [[ 3822,     0],
          [ 3851,     0]],

         [[ 4343,     0],
          [ 3624,     0]],

         [[ 4833,     0],
          [ 3777,     0]],

         [[ 5351,     0],
          [ 3705,     0]],

         [[ 5504,     0],
          [ 3699,     0]],

         [[ 5893,     0],
          [ 3546,     0]],

         [[ 6280,     0],
          [ 3551,     0]],

         [[ 6049,     0],
          [ 3547,     0]],

         [[ 6252,     0],
          [ 3476,     0]],

         [[ 5940,     0],
          [ 3380,     0]],

         [[ 6122,     0],
          [ 3440,     0]],

         [[ 6400,     0],
          [ 3359,     0]],

         [[ 6286,     0],
          [ 3411,     0]],

         [[ 5767,     0],
          [ 3285,     0]],

         [[ 5550,     0],
          [ 3381,     0]],

         [[ 4985,     0],
          [ 3326,     0]],

         [[ 3708,     0],
          [ 3339,     0]],

         [[ 3983,     0],
          [ 3299,     0]],

         [[ 7398,     0],
          [ 3355,     0]],

         [[13816,     0],
          [ 3301,     0]],

         [[13928,     0],
          [ 3245,     0]],

         [[13386,     0],
          [ 3150,     0]],

         [[10965,     0],
          [ 3261,     0]],

         [[14935,     0],
          [ 3196,     0]],

         [[15045,     0],
          [ 3172,     0]],

         [[15538,     0],
          [ 3118,     0]],

         [[16318,     0],
          [ 3106,     0]],

         [[13293,     0],
          [ 3069,     0]],

         [[17089,     0],
          [ 3055,     0]],

         [[20628,     0],
          [ 2997,     0]],

         [[20883,     0],
          [ 2960,     0]],

         [[20460,     0],
          [ 2901,     0]],

         [[20198,     0],
          [ 2889,     0]],

         [[21460,     0],
          [ 2826,     0]],

         [[23027,     0],
          [ 2857,     0]],

         [[22051,     0],
          [ 2807,     0]],

         [[22702,     0],
          [ 2786,     0]],

         [[16541,     0],
          [ 2727,     0]],

         [[24233,     0],
          [ 2722,     0]],

         [[21373,     0],
          [ 2687,     0]],

         [[25747,     0],
          [ 2639,     0]],

         [[26409,     0],
          [ 2456,     0]],

         [[24634,     0],
          [ 2541,     0]],

         [[27990,     0],
          [ 2526,     0]],

         [[28243,     0],
          [ 2494,     0]],

         [[27787,     0],
          [ 2432,     0]],

         [[25726,     0],
          [ 2384,     0]],

         [[12951,     0],
          [ 2294,     0]],

         [[ 1517,     0],
          [ 2289,     0]],

         [[26017,     0],
          [ 2263,     0]],

         [[26898,     0],
          [ 2268,     0]],

         [[ 5187,  4103],
          [ 2230,     0]],

         [[29570,     0],
          [ 2249,     0]],

         [[29075,     0],
          [ 2262,     0]],

         [[29087,     0],
          [ 2286,     0]]]], device='cuda:0'), 'mask': tensor([[[[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True,  True],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]]]], device='cuda:0'), 'type_ids': tensor([[[[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]]]], device='cuda:0')}}

In [18]:
## Testing TableembedderTorch module
from table_embedder.models.finetune_entitables_cp import TableEmbedder, TableEmbedderHypers
hypers = TableEmbedderHypers()
model = TableEmbedder(hypers).to('cuda:0')

In [21]:
loss, prob = model(table_info,indexed_headers, indexed_cells, None)

> /work/arnaik_umass_edu/tabbie_dev/table_embedder/models/finetune_entitables_cp.py(142)get_tabemb()
    140             if i == 1:
    141                 pdb.set_trace()
--> 142                 bert_data = TableUtil.add_cls_tokens(bert_header, bert_data, cls_row, cls_col, bs, n_rows, n_cols)
    143                 bert_data += row_pos_embs.expand((bs, n_cols, n_rows + 1, 768)).permute(0, 2, 1, 3).expand_as(bert_data)
    144                 bert_data += col_pos_embs.expand((bs, n_rows + 1, n_cols, 768)).expand_as(bert_data)



ipdb>  n


> /work/arnaik_umass_edu/tabbie_dev/table_embedder/models/finetune_entitables_cp.py(143)get_tabemb()
    141                 pdb.set_trace()
    142                 bert_data = TableUtil.add_cls_tokens(bert_header, bert_data, cls_row, cls_col, bs, n_rows, n_cols)
--> 143                 bert_data += row_pos_embs.expand((bs, n_cols, n_rows + 1, 768)).permute(0, 2, 1, 3).expand_as(bert_data)
    144                 bert_data += col_pos_embs.expand((bs, n_rows + 1, n_cols, 768)).expand_as(bert_data)
    145                 # table_mask = TableUtil.add_cls_mask(table_mask, table_info, bs, n_rows, n_cols, self.device)



ipdb>  row_pos_ids.shape


torch.Size([31])


...

58


ipdb>  self.hypers.num_max_row_pos


31


ipdb>  exit()


BdbQuit: 

In [ ]:
prob

In [4]:
# Load the torch Model
import torch
model_namedparam = torch.load('/work/arnaik_umass_edu/model_named.pt')
model_param = torch.load('/work/arnaik_umass_edu/model.pt')

In [1]:
#model_param

In [6]:
type(model_namedparam)

dict

In [2]:
import numpy as np

In [7]:
p = np.load('/work/arnaik_umass_edu/tabbie/data/clsrow.npy')

(768,)

In [5]:
table_info = [{'table_id': '1438042981460.12__CC-MAIN-20150728002301-00003-ip-10-236-191-2.ec2.internal__1387'
               , 'num_rows': 4
               , 'num_cols': 7
               , 'header': ['size', 'us', 'type', 'china', 'gaa', 'eu', 'years']
               , 'cell_labels': None
               , 'col_labels': [2, 4, 6]
               , 'table_labels': None
               , 'table_data_raw': [['xs', '6/7', '9.125', '235', '23', '36-37', '4']
                                    , ['s', '7/8', '9.5', '245', '24', '38-39', '5']
                                    , ['m', '8/9', '9.875', '255', '25', '39-40', '6']
                                    , ['l', '9/10', '10.375', '265', '26', '40-41', '7']]
               , 'table': [['size', 'us', 'type', 'china', 'gaa', 'eu', 'years']
                           , ['xs', '6/7', '9.125', '235', '23', '36-37', '4']
                           , ['s', '7/8', '9.5', '245', '24', '38-39', '5']
                           , ['m', '8/9', '9.875', '255', '25', '39-40', '6']
                           , ['l', '9/10', '10.375', '265', '26', '40-41', '7']]}]

In [6]:
indexed_headers = {'bert': {'token_ids': tensor([[[  101,  2946,   102],
         [  101,  2149,   102],
         [  101,  2828,   102],
         [  101,  2859,   102],
         [  101, 19930,   102],
         [  101,  7327,   102],
         [  101,  2086,   102]]], device='cuda:0')
, 'mask': tensor([[[True, True, True],
         [True, True, True],
         [True, True, True],
         [True, True, True],
         [True, True, True],
         [True, True, True],
         [True, True, True]]], device='cuda:0')
, 'type_ids': tensor([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]], device='cuda:0')}}

In [7]:
indexed_cells = {'bert': {'token_ids': tensor([[[[  101,  1060,  2015,   102,     0],
          [  101,  1020,  1013,  1021,   102],
          [  101,  1023,  1012,  8732,   102],
          [  101, 17825,   102,     0,     0],
          [  101,  2603,   102,     0,     0],
          [  101,  4029,  1011,  4261,   102],
          [  101,  1018,   102,     0,     0]],

         [[  101,  1055,   102,     0,     0],
          [  101,  1021,  1013,  1022,   102],
          [  101,  1023,  1012,  1019,   102],
          [  101, 21005,   102,     0,     0],
          [  101,  2484,   102,     0,     0],
          [  101,  4229,  1011,  4464,   102],
          [  101,  1019,   102,     0,     0]],

         [[  101,  1049,   102,     0,     0],
          [  101,  1022,  1013,  1023,   102],
          [  101,  1023,  1012, 27658,   102],
          [  101, 20637,   102,     0,     0],
          [  101,  2423,   102,     0,     0],
          [  101,  4464,  1011,  2871,   102],
          [  101,  1020,   102,     0,     0]],

         [[  101,  1048,   102,     0,     0],
          [  101,  1023,  1013,  2184,   102],
          [  101,  2184,  1012, 18034,   102],
          [  101, 20549,   102,     0,     0],
          [  101,  2656,   102,     0,     0],
          [  101,  2871,  1011,  4601,   102],
          [  101,  1021,   102,     0,     0]]]], device='cuda:0')
, 'mask': tensor([[[[ True,  True,  True,  True, False],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True, False, False],
          [ True,  True,  True, False, False],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True, False, False]],

         [[ True,  True,  True, False, False],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True, False, False],
          [ True,  True,  True, False, False],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True, False, False]],

         [[ True,  True,  True, False, False],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True, False, False],
          [ True,  True,  True, False, False],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True, False, False]],

         [[ True,  True,  True, False, False],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True, False, False],
          [ True,  True,  True, False, False],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True, False, False]]]], device='cuda:0')
, 'type_ids': tensor([[[[0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]],

         [[0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]],

         [[0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]],

         [[0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]]]], device='cuda:0')}}